In [1]:
BUCKET_NAME='test-xwh01'
OBS_BASE_PATH=BUCKET_NAME

from modelarts.session import Session
session = Session()

session.download_data(bucket_path="test-xwh01/test-machine/ratings.dat", path="./ratings.dat")
session.download_data(bucket_path="test-xwh01/test-machine/users.dat", path="./users.dat")
session.download_data(bucket_path="test-xwh01/test-machine/movies.dat", path="./movies.dat")
# import same usefull libraries
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
movie_rating = pd.read_csv('./ratings.dat')

Successfully download file test-xwh01/test-machine/ratings.dat from OBS to local ./ratings.dat
Successfully download file test-xwh01/test-machine/users.dat from OBS to local ./users.dat
Successfully download file test-xwh01/test-machine/movies.dat from OBS to local ./movies.dat


In [2]:
movie_rating_cols = ['user_id',  'movie_id', 'rating', 'unix_tiemstamp'] 
movie_rating = pd.read_csv('./ratings.dat',sep = '::', names = movie_rating_cols, parse_dates = True)
movie_rating.head()

/home/ma-user/anaconda3/envs/XGBoost-Sklearn/lib/python3.6/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


,user_id,movie_id,rating,unix_tiemstamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
movie_rating.nunique()

user_id             6040
movie_id            3706
rating                 5
unix_tiemstamp    458455
dtype: int64

In [4]:
user=pd.read_csv('./users.dat',sep='::',names=['UserID','Gender','Age','Occupation','Zip-code'])
user.head()

/home/ma-user/anaconda3/envs/XGBoost-Sklearn/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
user.shape

(6040, 5)

In [6]:
movie=pd.read_csv('./movies.dat',sep='::',names=['MovieID','Title','Genres'])
movie.head()

/home/ma-user/anaconda3/envs/XGBoost-Sklearn/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movie.shape

(3883, 3)

In [8]:
movie_rating.drop("unix_tiemstamp",inplace = True, axis = 1)
movie_rating.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [9]:
# 拆分训练集与数据集
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(movie_rating,test_size = 0.15)

In [10]:
# 创建用户电影矩阵
movie_rating_matrix = train_data.pivot_table(index = ['movie_id'], columns = ['user_id'],
                                              values = 'rating').reset_index(drop = True)
movie_rating_matrix.fillna(0, inplace = True)
cmu = movie_rating_matrix
cmu.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,5.0,5.0,...,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# 利用余弦算法获得电影相似度矩阵
movie_similarity = 1 - pairwise_distances(movie_rating_matrix.values, metric = "cosine")
np.fill_diagonal(movie_similarity,0)
# 将相似度填回相似矩阵
movie_rating_matrix = pd.DataFrame(movie_similarity)
movie_rating_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,3674,3675,3676,3677,3678,3679,3680,3681,3682,3683
0,0.000000,0.348915,0.219256,0.138802,0.215607,0.291200,0.267863,0.112837,0.092735,0.319398,...,0.092122,0.033030,0.084535,0.065109,0.043173,0.266591,0.147270,0.081525,0.044069,0.161325
1,0.348915,0.000000,0.207196,0.153716,0.226281,0.231499,0.242832,0.146032,0.123796,0.330210,...,0.061542,0.023797,0.067592,0.094687,0.074170,0.178306,0.120458,0.072911,0.012238,0.108352
2,0.219256,0.207196,0.000000,0.152017,0.261219,0.161221,0.241753,0.064557,0.106937,0.210850,...,0.042044,0.057634,0.037695,0.074686,0.054287,0.161447,0.082590,0.056599,0.012209,0.082696
3,0.138802,0.153716,0.152017,0.000000,0.241165,0.106091,0.212059,0.040616,0.048904,0.117646,...,0.054455,0.000000,0.002423,0.022807,0.015372,0.097685,0.070356,0.016087,0.021977,0.084445
4,0.215607,0.226281,0.261219,0.241165,0.000000,0.115742,0.269268,0.086547,0.114554,0.212286,...,0.012456,0.054297,0.043032,0.031546,0.018717,0.146647,0.080016,0.047228,0.013008,0.088156


In [12]:
# 获得TOP-N推荐
user_inp = "Toy Story (1995)"
inp = movie[movie['Title'] == user_inp].index.tolist()
inp = inp[0]
movie['similarity'] = movie_rating_matrix.iloc[inp]
movie.columns = ['MovieID','Title','Genres','similarity']
movie.head()

,MovieID,Title,Genres,similarity
0,1,Toy Story (1995),Animation|Children's|Comedy,0.000000
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.348915
2,3,Grumpier Old Men (1995),Comedy|Romance,0.219256
3,4,Waiting to Exhale (1995),Comedy|Drama,0.138802
4,5,Father of the Bride Part II (1995),Comedy,0.215607


In [13]:
# 测试集合构造
rating_test_matrix = test_data.pivot_table(index = ['movie_id'],columns = ['user_id'],
                                values = 'rating').reset_index(drop = True)
rating_test_matrix.fillna(0,inplace = True)
cmu_test = rating_test_matrix
cmu_test.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# 测试集构造物品相似矩阵
movie_similarity_test = 1 - pairwise_distances(rating_test_matrix.values, metric = "cosine")
np.fill_diagonal(movie_similarity_test,0)
rating_matrix_test = pd.DataFrame(movie_similarity_test)
rating_matrix_test.head()

,0,1,2,3,4,5,6,7,8,9,...,3427,3428,3429,3430,3431,3432,3433,3434,3435,3436
0,0.000000,0.057780,0.037752,0.016519,0.050142,0.046348,0.007175,0.000000,0.000000,0.064348,...,0.0,0.0,0.0,0.005751,0.0,0.042614,0.008377,0.000000,0.0,0.023456
1,0.057780,0.000000,0.061666,0.000000,0.044321,0.065718,0.034728,0.000000,0.034241,0.047048,...,0.0,0.0,0.0,0.006524,0.0,0.012643,0.014253,0.026258,0.0,0.016695
2,0.037752,0.061666,0.000000,0.018283,0.021622,0.021641,0.056914,0.000000,0.013920,0.041348,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.009271,0.010675,0.0,0.057267
3,0.016519,0.000000,0.018283,0.000000,0.015768,0.010521,0.000000,0.039036,0.000000,0.049846,...,0.0,0.0,0.0,0.000000,0.0,0.023813,0.000000,0.000000,0.0,0.022274
4,0.050142,0.044321,0.021622,0.015768,0.000000,0.027997,0.070206,0.000000,0.000000,0.054583,...,0.0,0.0,0.0,0.000000,0.0,0.038722,0.000000,0.000000,0.0,0.000000


In [15]:
# 测试集TOP-N推荐
user_inp = "Toy Story (1995)"
inp = movie[movie['Title'] == user_inp].index.tolist()
inp = inp[0]
movie['similarity'] = rating_matrix_test.iloc[inp]
movie.columns = ['MovieID','Title','Genres','similarity']
movie.head()

,MovieID,Title,Genres,similarity
0,1,Toy Story (1995),Animation|Children's|Comedy,0.000000
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.057780
2,3,Grumpier Old Men (1995),Comedy|Romance,0.037752
3,4,Waiting to Exhale (1995),Comedy|Drama,0.016519
4,5,Father of the Bride Part II (1995),Comedy,0.050142


In [16]:
# 训练集预测，求得RMSE
# 将dataframe形式的用户物品相似矩阵转为numpy 数组
rating_matrix_array = movie_rating_matrix.values
user_item_pre = rating_matrix_array.dot(movie_similarity) / np.array([np.abs(movie_similarity).sum(axis = 1)])

pre_flatten = user_item_pre
rating_matrix_flatten = rating_matrix_array
from sklearn.metrics import mean_squared_error
from math import sqrt
error_train = sqrt(mean_squared_error(pre_flatten, rating_matrix_flatten))
print(error_train)


0.05267091294769178


In [17]:
# 测试集预测，求得RMSE
rating_test_matrix_array = rating_matrix_test.values
user_item_test_pre = rating_test_matrix_array.dot(movie_similarity_test) / np.array([np.abs(movie_similarity_test).sum(axis = 1)])

test_pre_flatten = user_item_test_pre
test_rating_matrix_flatten = rating_test_matrix_array
error_test = sqrt(mean_squared_error(test_pre_flatten, test_rating_matrix_flatten))
print(error_test)

0.023443807117521632
